In [22]:
import os
import sys
import numpy as np
import getpass
import pandas as pd
from arcgis import GIS

user = getpass.getuser()

sys.path.insert(0, f"/Users/{user}/Documents/GitHub/Housing-Analysis")

from housing_utils import *

In [2]:
# A public ArcGIS Online account is required to pull data from ArcGIS Online using the arcgis python api.
# Link to create an account: https://doc.arcgis.com/en/arcgis-online/get-started/create-account.htm
password = os.environ.get("AGOL_PUBLIC_PASSWORD")
gis = GIS(username="joshua.croff@gmail.com", password=password)

# Read data

In [3]:
# read housing sites inventory
sites_df = pd.read_excel("Data/AppendixB4.xlsx", sheet_name=1, header=1)

In [4]:
# read cycle 4 and 5 housing element sites inventory
url ="https://services3.arcgis.com/i2dkYWmb4wHvYPda/arcgis/rest/services/regional_housing_need_assessment_sites/FeatureServer/0"
prev_sites_df = pull_geotable_agol(base_url=url, client=gis, reproject_to_analysis_crs=False)


Breaking feature service layer IDs into 188 chunks


## Merge data

In [10]:
# create dataframe that only includes sites in San Francisco
prev_sites_sf_df = prev_sites_df.query("jurisdict == 'San Francisco'").copy()

In [14]:
# fix apn formatting from MTC/ABAG data to match SF data
prev_sites_sf_df["apn_fmt"] = prev_sites_sf_df["apn"].str.replace("/","-")

In [33]:
# cast all columns to lowercase
sites_df.columns = sites_df.columns.str.lower()

In [38]:
# rename columns
sites_df.rename(columns={"mapblklot": "apn_fmt"}, inplace=True)

In [39]:
# merge sites dataframes
sites_merge_df = pd.merge(sites_df, prev_sites_sf_df, on="apn_fmt", how="outer", indicator=True)

In [40]:
# check merge results
sites_merge_df["_merge"].value_counts() 

_merge
left_only     115067
both            8396
right_only      2366
Name: count, dtype: int64

In [41]:
# create flag binary flag columns to indicate if site used in previous housing element
sites_merge_df["sf_used_in_previous_he"] = np.where(
    sites_merge_df["id_last2"] == "Used in Prior Housing Element - Non-Vacant", 1, 0
)

sites_merge_df["abag_used_in_previous_he"] = np.where(sites_merge_df["rhnacyc"].notnull(), 1, 0)

In [49]:
# rename columns for export
sites_merge_df.rename(
    columns={
        "jurisdict_x": "jurisdiction",
        "rhnacyc": "abag_rhna_cycle",
        "rhnayrs": "abag_rhna_years",
        "genplan": "abag_general_plan",
        "zoning": "abag_zoning",
        "allowden": "abag_allowable_density",
    },
    inplace=True,
)

In [50]:
out_cols = [
    "jurisdiction",
    "address",
    "zip5",
    "apn_fmt",
    "con_sites",
    "ex_gp_des",
    "ex_zoning",
    "min_dens",
    "max_dens",
    "acres",
    "ex_use_vac",
    "infra",
    "public",
    "site_stat",
    "id_last2",
    "li",
    "mod",
    "amod",
    "capacity",
    "opt1",
    "opt2",
    "abag_general_plan",
    "abag_zoning",
    "abag_allowable_density",
    "abag_rhna_cycle",
    "abag_rhna_years",
    "sf_used_in_previous_he",
    "abag_used_in_previous_he",
]
final_sites_df = sites_merge_df.query("_merge == 'both' and sf_used_in_previous_he == 0")[out_cols].copy()

In [52]:
final_sites_df.shape

(1337, 28)

In [51]:
# export sites inventory
final_sites_df.to_csv("Data/sf_recycled_sites.csv", index=False)